### Step 0
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

In [1]:
import pandas as pd    
energy = pd.read_excel('/home/hager/Desktop/ITI_Python_for_Data_Management/Task 1/data/Energy Indicators.xls',
names=['Country','Energy Supply', 'Energy Supply per Capita', '% Renewable'],  
usecols="C,D,E,F", skiprows=17, skipfooter=38 , 
na_values='...')   

energy['Country'] = energy['Country'].str.replace('\d+', '')

energy['Country'] = energy['Country'].str.replace(r" \(.*\)","") 


energy['Energy Supply'] = 1000000 * energy['Energy Supply']   
energy['Country'] = energy['Country'].replace({'Republic of Korea' : 'South Korea',
'United States of America' : 'United States',
'United Kingdom of Great Britain and Northern Ireland':'United Kingdom',
'China, Hong Kong Special Administrative Region':'Hong Kong'}  ) 


### Step 1
<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

In [2]:
GDP = pd.read_csv ('/home/hager/Desktop/ITI_Python_for_Data_Management/Task 1/data/world_bank.csv' , header=4 )  
GDP.replace({'Country Name' : 
{ "Korea, Rep.": "South Korea","Iran, Islamic Rep.": "Iran","Hong Kong SAR, China": "Hong Kong"}} ,
inplace=True)  
GDP.rename(columns={'Country Name': 'Country'} , inplace=True)  


### Step 2
Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

In [3]:
ScimEn = pd.read_excel('/home/hager/Desktop/ITI_Python_for_Data_Management/Task 1/data/scimagojr-3.xlsx' )  

### Step 3
Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This step should yeild a DataFrame with 20 columns and 15 entries.*

In [18]:
# merge1 = pd.merge(ScimEn[:15],energy,how="inner",on="Country") 
merge1 = pd.merge(ScimEn,energy,how="inner",on="Country")
merge1 = merge1[merge1['Rank'] <= 15] 
merge2 = pd.merge(merge1,GDP[['Country','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]
,how='inner',on='Country')  
merge2 = merge2 [['Country' ,'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 
'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita',
'% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
merge2.set_index("Country", inplace=True) 


### Step 4
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This step should yield a single number.*

In [5]:
merge11 = pd.merge(ScimEn,energy) 
merge22 = pd.merge(merge11,GDP )   
# 162 rows × 70 columns  
# Solution is 162 - 15 = 147 row 

#OR 
merge111 = pd.merge(ScimEn[15:],energy) 
merge222 = pd.merge(merge111,GDP )   
# 147 rows × 70 columns

### Step 5

#### Answer the following questions in the context of only the top 15 countries by Scimagojr Rank 


What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This step should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [19]:
# Step 5
avgGDP = merge2.mean(axis=1 , skipna = True ).sort_values(ascending=False) 

### Step  6
What is the mean `Energy Supply per Capita`?

*This step should return a single number.*

In [7]:
# Step 6
merge2["Energy Supply per Capita"].mean()

157.6

### Step 7
What country has the maximum % Renewable and what is the percentage?

*This step should return a tuple with the name of the country and the percentage.*

In [8]:
# Step 7 
(merge2['% Renewable'].idxmax() , merge2['% Renewable'].max() )

('Brazil', 69.64803)

### Step 8
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This step should return a tuple with the name of the country and the ratio.*

In [9]:
# Step 8
merge2['Ratio'] = merge2['Self-citations']/merge2['Citations'] 
(merge2['Ratio'].idxmax() , merge2['Ratio'].max() )

('China', 0.6893126179389422)

### Step 9
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This step should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [20]:
# Step 9
var = merge2["% Renewable"].median(axis=0) 
merge2['HighRenew'] = merge2['% Renewable'].apply(lambda x : 1 if x >= var else 0 ) 
merge2.sort_values('Rank')['HighRenew']

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Step 10
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [11]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}



In [14]:
merge2['population']=merge2['Energy Supply']/merge2['Energy Supply per Capita']
merge2.rename(index=ContinentDict,inplace=True)  
merge2.reset_index(inplace = True) 
functions = ['size', 'sum', 'mean', 'std']
result = merge2[['Country','population']].groupby('Country').agg(functions)
result

population                                          
                    size           sum          mean           std
Country                                                           
Asia                   5  2.898666e+09  5.797333e+08  6.790979e+08
Australia              1  2.331602e+07  2.331602e+07           NaN
Europe                 6  4.579297e+08  7.632161e+07  3.464767e+07
North America          2  3.528552e+08  1.764276e+08  1.996696e+08
South America          1  2.059153e+08  2.059153e+08           NaN